In [ ]:
import numpy as np
import random
import temet
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.sim_data_helpers import get_data_from_header
from util.cosmo_helpers import redshift_wavelength_forward
from util.spectra_helpers import load_spectra_from_boxes, patch_spectra_together, add_noise_to_spectrum

In [ ]:
n_spectra_each_box = 1000

gps_list = [i for i in range(50)]
base_path = "/vera/u/jerbo/my_ptmp/L25n128_suite_var"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]
spectrum_num = random.sample(range(0, 10000), n_spectra_each_box)

Ly_alpha_0 = 1215.67

redshifts = []
Ly_alpha_z = []
Ly_alpha_box = []
for current_gp in gp_paths:
    sim = temet.sim(current_gp, redshift=2.0)
    dz = sim.dz
    z = get_data_from_header(current_gp, 3, param="Redshift")
    z = z + dz
    redshifts.append(z)

    Ly_alpha_z.append(redshift_wavelength_forward(z, Ly_alpha_0))
    Ly_alpha_box.append(redshift_wavelength_forward(z - dz, Ly_alpha_0))

wavelengths, taus, params = load_spectra_from_boxes(gp_paths, spectrum_num, y_param='tau_HI_1215')

In [ ]:
taus=np.array(taus)
fluxes = np.exp(-taus)

_ , flux_test, _ = load_spectra_from_boxes(gp_paths, spectrum_num, y_param='flux')
flux_test = np.array(flux_test)

assert np.array_equal(flux_test, fluxes), "Mistake in calculation of data loading!"
        

In [ ]:
print(taus[10][0])
print(fluxes[10][0])

In [ ]:

for i in range(10, len(wavelengths)-30):
    plt.plot(wavelengths[0], fluxes[i][0], alpha=0.6)

    plt.plot([Ly_alpha_z[i], Ly_alpha_z[i]],[0.3, 1.1], alpha=0.5, linestyle="--")

# plt.annotate(r"$Ly \alpha$", [Ly_alpha_0-1.5, 1.05], rotation=90))

plt.xlabel(r"Rest-frame wavelength [$\AA$]")
plt.ylabel(r"Relative flux ")
plt.xlim([3620, 3710])
plt.show()

In [ ]:
gp_index_to_use = 5
n_rows = 6
n_cols = 6

spectrum_numbers = random.sample(range(0, 1001), n_rows*n_cols)

In [ ]:
# note: the way this works is extremely inefficient at the moment. should change this if I want to run something similar repeatedly
fig, ax = plt.subplots(figsize=(20,20))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(n_rows, n_cols)
axes = []

for j in range(len(spectrum_numbers)): 

    row = j//max([n_rows, n_cols])
    col = j%max([n_rows, n_cols])
    
    ax = fig.add_subplot(gs[row, col])

    ax.plot(wavelengths[gp_index_to_use], fluxes[gp_index_to_use][j])

    ax.plot([Ly_alpha_z[gp_index_to_use], Ly_alpha_z[gp_index_to_use]],[-0.1, 1.1], c="black", linestyle="--")
    ax.annotate(r"$Ly \alpha$", [Ly_alpha_z[gp_index_to_use]+3.5, 1.05], rotation=0)

    ax.plot([Ly_alpha_box[gp_index_to_use], Ly_alpha_box[gp_index_to_use]],[-0.1, 1.1], c="black", linestyle="--")

    ax.set_xlim([3620, 3710])

    ax.text(3620+5, 0.0, f"#{spectrum_numbers[j]}", va='bottom', ha='left')
    
    axes.append(ax)

plt.savefig("plots/single_spectra_galery.pdf", format="PDF")
plt.show()

In [ ]:
test_gp = 34
left_cut = 3550
right_cut = 3950
test_gp_path = f"/vera/u/jerbo/my_ptmp/L25n128_suite_var/gridpoint{test_gp}/"
wavelengths, patched_spectra, specs_to_plot, plot_specs = patch_spectra_together(test_gp_path, [10, 20, 100], left_cut, right_cut, return_plot_vars=True)

In [ ]:
wavelengths, flux, params = load_spectra_from_boxes([test_gp_path], [i for i in range(10000)], y_param='flux')
wavelengths = wavelengths[0]
flux = flux[0]

In [ ]:
specs_to_plot = specs_to_plot[1]
plot_specs = plot_specs[len(specs_to_plot):len(specs_to_plot)*2]
patched_spectrum = patched_spectra[1]

left_edge = Ly_alpha_box[test_gp]
right_edge = Ly_alpha_z[test_gp]

In [ ]:
def get_distinct_colors(n):
    tab20 = cm.get_cmap("tab20").colors
    base = tab20[::2]  # 10 stark unterscheidbare Farben
    # Bei Bedarf loop/expandieren:
    return [base[i % len(base)] for i in range(n)]


colors_plot = get_distinct_colors(len(specs_to_plot))

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(2, len(specs_to_plot))
axes = []

for j in range(len(specs_to_plot)): 
    row = 0
    col = j

    ax = fig.add_subplot(gs[row, col])

    ax.plot(wavelengths, flux[specs_to_plot[j]], color=colors_plot[j])

    ax.text(left_edge+1, 0.01, f"#{specs_to_plot[j]}", va='bottom', ha='left') 

    ax.set_ylim([0, 1.1])
    ax.set_xlim([left_edge, right_edge])

    if col == 0:
        ax.set_ylabel("Realtive Flux")
    else:   
        ax.set_yticklabels([])

    axes.append(ax)

ax = fig.add_subplot(gs[1, :])

for j in range(len(plot_specs)):
    ax.plot(wavelengths, plot_specs[j], color=colors_plot[j])
    ax.fill_between(wavelengths, plot_specs[j], 1, alpha=0.3, color=colors_plot[j])
    ax.set_ylim(0, 1.1)
    ax.set_xlim([left_cut, right_cut])
    axes.append(ax)

# ax = fig.add_subplot(gs[2, :])
ax.plot(wavelengths, patched_spectrum, color="black", alpha=1)
ax.set_xlim([left_cut, right_cut])
ax.set_ylim([0, 1.1])

ax.set_xlabel(r"Wavelength [$\AA$]")
ax.set_ylabel("Relative Flux")

plt.savefig("plots/spectrum_augmentation.pdf", format="PDF")
plt.show()

In [ ]:
noisy_specs = []
for spec in patched_spectra:
    noisy_specs.append(add_noise_to_spectrum(spec, 0.2, distr='normal'))


In [ ]:
fig, ax = plt.subplots(figsize=(13, 10))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(len(noisy_specs), 1, hspace=0)
axes = []

for i in range(len(noisy_specs)):
    ax = fig.add_subplot(gs[i, 0])

    ax.plot(wavelengths, noisy_specs[i], label="spectrum + noise")
    ax.plot(wavelengths, patched_spectra[i], color="black", label="original spectrum")
    ax.set_xlim([left_cut, right_cut])
    ax.set_ylabel("Relative Flux")
    ax.legend(loc="upper left")
    ax.tick_params(axis='x', direction='in')
    if i == len(noisy_specs)-1:
        ax.set_xlabel(r"Wavelength [$\AA$]")
    if i < len(noisy_specs)-1:
        ax.tick_params(labelbottom=False)

    axes.append(ax)

# fig.supxlabel(r"Wavelength [$\AA$]")
# fig.supylabel("Relative Flux")

plt.savefig("plots/noise_spectra.pdf", format="PDF")
plt.show()